### Terminology

| English | 中文 |
|:---|:---|
| Regular session / normal hours | 日盤 |
| After-hours session | 夜盤 |


### Notes
- There are 1/7 of tick data is duplicate.
- number of records of 內盤/外盤/無法判斷
```
print(ticks['tick_type'].value_counts())
tick_type
2    3009917
1    2941645
0     222400
Name: count, dtype: int64
```
- 日盤第一個 tick 在 08:45:00 之後，最後一個 tick （絕大部分）在 13:45:00 之前
    - 有 8 筆是在 13:45:00 之後。由於筆數少且 volume 小，所以直接忽略。資料如下：

        ```
                                   close  volume  bid_price  bid_volume  ask_price  ask_volume  tick_type  
        2024-08-08 13:45:00.026  20696.0       1    20691.0          17    20696.0           1          1  
        2024-08-12 13:45:00.075  21677.0       1    21673.0           1    21677.0          40          1  
        2024-08-27 13:45:00.066  22159.0       4    22146.0           6    22159.0           6          1  
        2024-08-27 13:45:00.067  22146.0       2    22146.0           6    22159.0           6          2  
        2024-08-28 13:45:00.016  22401.0       1    22401.0           1    22404.0           4          2 
        2024-10-09 13:45:00.111  22702.0       1    22700.0          40    22702.0           6          1  
        2024-10-22 13:45:00.004  23629.0       1    23629.0           7    23633.0          62          2  
        2024-10-22 13:45:00.006  23633.0       5    23629.0           7    23633.0          62          1 
        ```
        
    - placeholder

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
def remove_duplicates(ticks):
    result = ticks.reset_index(names='timestamp')
    # For debug only
    # result[result.duplicated()].to_csv('duplicates.csv')
    result = result.drop_duplicates()
    result = result.set_index('timestamp')
    return result

In [3]:
# make datetime as index of dataframe
ticks = pd.read_csv(f"TXF_tick_startFrom20240801.csv", index_col=0)

# drop duplicate tick data
ticks = remove_duplicates(ticks)

# convert index type from object to datetime64[ns]
ticks.index = pd.to_datetime(ticks.index)

In [4]:
def get_date(my_datetime):
    return my_datetime.astype('datetime64[D]')

In [5]:
def create_DHLD(ticks, session_type, large_threshold, *, SHLD_toggle=0):
    """
        Params
            ticks: dataframe, all tick data
            session_type: {'day', 'night'}, indicates `time_range`
                The format of `time_range`: tuple, (start_time, end_time), indicating the time range of data should be processed
            large_threshold: integer, all volume >= `large_threshold` should be calculated in DHLD
            SHLD_toggle: if `SHLD_toggle=1` then generate SHLD instead, which means calculate data by using 
                    data except for DHLD
            
        Return Values
            DHLD_min: DHLD in minute
            DHLD_sec: DHLD in second
    """
    
    TIME_RANGE_DAY = ('08:45', '13:45')
    TIME_RANGE_NIGHT = ('15:00', '05:00')
    ADJUSTED_TIME_RANGE_NIGHT = ('10:00', '23:59:59.999999') # for night session processing
    if session_type == 'day':
        time_range = TIME_RANGE_DAY
    elif session_type == 'night':
        time_range = TIME_RANGE_NIGHT

    ticks_calc = ticks.between_time(*time_range)

    # Since the time range of after-hours session cross over to the next day,
    # which leads to 2 dates in one session. Rewind 5 hours to avoid this issue.
    if session_type == 'night':
        ticks_calc = ticks_calc.shift(periods=-5, freq='h')

    # Remove the first tick from every day
    first_ticks_everyday = ticks_calc.groupby(ticks_calc.index.date).head(1)
    # print(first_ticks_everyday)
    ticks_calc = ticks_calc.drop(first_ticks_everyday.index)

    # Select the record whose column 'volume' >= 'large_threshold'
    # and split the dataframe to which will be added/subtracted
    # Note: tick_type = 0/1/2 means 無法判定/外盤/內盤
    if SHLD_toggle == 0:
        ticks_calc = ticks_calc[ticks_calc['volume'] >= large_threshold]
    elif SHLD_toggle == 1:
        ticks_calc = ticks_calc[ticks_calc['volume'] < large_threshold]
    
    ticks_add = ticks_calc[ticks_calc['tick_type'] == 1]
    ticks_substract = ticks_calc[ticks_calc['tick_type'] == 2]

    # Note: use `between_time()` in the last,
    # due to `.groupby().sum()` will create empty record beyond `time_range`
    a = ticks_add['volume'].groupby([pd.Grouper(freq='s')]).sum()
    b = ticks_substract['volume'].groupby([pd.Grouper(freq='s')]).sum()

    if session_type == 'day':
        a = a.between_time(*time_range)
        b = b.between_time(*time_range)
    elif session_type == 'night':
        a = a.between_time(*ADJUSTED_TIME_RANGE_NIGHT)
        b = b.between_time(*ADJUSTED_TIME_RANGE_NIGHT)

    # Use `fill_value=0` to prevent the null record in some seconds
    c = a.sub(b, fill_value=0)

    # Calculate DHLD in second
    DHLD_sec = c.groupby([pd.Grouper(freq='D')]).cumsum()

    # Get DHLD in minute by using the last record in DHLD_sec
    # and set second (of time) field to 0
    DHLD_min = DHLD_sec.groupby([pd.Grouper(freq='min')]).tail(1)
    DHLD_min.index = DHLD_min.index.floor('min')

    # Shift back (fast forward 5 hours) to get correct timestamps in after-hours seesion
    if session_type == 'night':
        DHLD_sec = DHLD_sec.shift(periods=5, freq='h')
        DHLD_min = DHLD_min.shift(periods=5, freq='h')

    return DHLD_min, DHLD_sec

In [6]:
# test

import os
path = 'test'
if not os.path.isdir(path):
   os.makedirs(path)

DHLD_night_min, DHLD_night_sec = create_DHLD(ticks, 'day', 2)
DHLD_night_min.to_csv(f'./test/DHLD_day_min_2.csv')
DHLD_night_sec.to_csv(f'./test/DHLD_day_sec_2.csv')

DHLD_night_min, DHLD_night_sec = create_DHLD(ticks, 'day', 2, SHLD_toggle=1)
DHLD_night_min.to_csv(f'./test/SHLD_day_min_2.csv')
DHLD_night_sec.to_csv(f'./test/SHLD_day_sec_2.csv')

DHLD_night_min, DHLD_night_sec = create_DHLD(ticks, 'night', 2)
DHLD_night_min.to_csv(f'./test/DHLD_night_min_2.csv')
DHLD_night_sec.to_csv(f'./test/DHLD_night_sec_2.csv')

DHLD_night_min, DHLD_night_sec = create_DHLD(ticks, 'night', 2, SHLD_toggle=1)
DHLD_night_min.to_csv(f'./test/SHLD_night_min_2.csv')
DHLD_night_sec.to_csv(f'./test/SHLD_night_sec_2.csv')

In [7]:
# Generate serveral DHLDs with different thresholds

thres_range = range(1, 20)

import os
if not os.path.isdir('DHLD'):
   os.makedirs('DHLD')

if not os.path.isdir('SHLD'):
   os.makedirs('SHLD')

for thres in thres_range:
    DHLD_day_min, DHLD_day_sec = create_DHLD(ticks, 'day', thres)
    DHLD_day_min.to_csv(f'./DHLD/DHLD_day_min_{thres}.csv')
    DHLD_day_sec.to_csv(f'./DHLD/DHLD_day_sec_{thres}.csv')

    DHLD_night_min, DHLD_night_sec = create_DHLD(ticks, 'night', thres)
    DHLD_night_min.to_csv(f'./DHLD/DHLD_night_min_{thres}.csv')
    DHLD_night_sec.to_csv(f'./DHLD/DHLD_night_sec_{thres}.csv')

    DHLD_day_min, DHLD_day_sec = create_DHLD(ticks, 'day', thres, SHLD_toggle=1)
    DHLD_day_min.to_csv(f'./SHLD/SHLD_day_min_{thres}.csv')
    DHLD_day_sec.to_csv(f'./SHLD/SHLD_day_sec_{thres}.csv')

    DHLD_night_min, DHLD_night_sec = create_DHLD(ticks, 'night', thres, SHLD_toggle=1)
    DHLD_night_min.to_csv(f'./SHLD/SHLD_night_min_{thres}.csv')
    DHLD_night_sec.to_csv(f'./SHLD/SHLD_night_sec_{thres}.csv')


KeyboardInterrupt



In [ ]:
def ticks_resample(ticks, session, period):
    """
        Params
            ticks: DataFrame, tick data with datetime index
            session: {'day', 'night'}
            period: {'min', 'sec'}
            
        Return
            resampled_ticks: ticks data which is resampled based on given `session` and `frequency`
    """

    TIME_RANGE_DAY = ('08:45', '13:45')
    TIME_RANGE_NIGHT = ('15:00', '05:00')

    # time_range_option
    if session == 'day':
        time_range_option = TIME_RANGE_DAY
    elif session == 'night':
        time_range_option = TIME_RANGE_NIGHT

    # `period_option`
    if period == 'min':
        period_option = 'min'
    elif period == 'sec':
        period_option = 's'

    resampled_ticks = ticks.between_time(*time_range_option)

    # Resample tick data accroding to `freq`
    # Note: I think df.resample() would be better here, but I don't know how to use. So here's just a workaround.
    resampled_ticks = resampled_ticks.groupby([pd.Grouper(freq=period_option)]).tail(1)
    resampled_ticks.index = resampled_ticks.index.floor(period_option)
    resampled_ticks = resampled_ticks.shift(periods=1, freq=period_option)

    return resampled_ticks

In [ ]:
# Calculate correlation
def correlation(ticks, session, period, thresholds, delay):
    """
        Params:
            ticks: DataFrame, tick data with datetime index
            session: {'day', 'night'}
            period: {'min', 'sec'}
            threshold: DHLD with threshold from 1 to `threshold` would be count into calculation
            delay: int or 2-tuple, 
                   calculate the correlation of delay from 0 to delay or [start, end] if 2-tuple is (start, end)
            
        Return
            resampled_ticks: ticks data which is resampled based on given `session` and `frequency`
    """

    # `period_option`
    if period == 'min':
        period_option = 'min'
    elif period == 'sec':
        period_option = 's'

    ticks_resampled = ticks_resample(ticks, session, period)
    
    result = pd.DataFrame()
    
    for threshold in range(1, thresholds+1):
        DHLD = pd.read_csv(f"./DHLD/DHLD_{session}_{period}_{threshold}.csv", index_col=0)
        DHLD.index = pd.to_datetime(DHLD.index)
        
        # Normalization in every day
        DHLD = DHLD.groupby([pd.Grouper(freq='D')]).transform(lambda x: (x-x.mean()) / x.std())
        
        corr_matrix = pd.DataFrame()
        corr_matrix[f'DHLD_{session}_{period}_{threshold}'] = DHLD

        delay_range = ()  # tuple
        if isinstance(delay, int):
            delay_range = (0, delay+1)
        elif isinstance(delay, tuple) and len(delay)==2:
            start, end = delay
            delay_range = (start, end+1)
        else:
            raise TypeError('`delay` should be a integer or a 2-tuple')
            
        # Todo: this for-loop could be only done 1 time for all circumstance
        #       especially in calculating large range of delays
        for delay in range(*delay_range):
            # Shift and normalization in every day
            ticks_resampled_shift = ticks_resampled.shift(periods=delay, freq=period_option)
            ticks_resampled_shift = ticks_resampled_shift.groupby([pd.Grouper(freq='D')]).transform(lambda x: (x-x.mean()) / x.std())
            corr_matrix[f"FuturesClose_{session}_{period}_{delay}"] = ticks_resampled_shift
    
        # Ignore NA while calculating correlation
        corr_matrix = corr_matrix.corr(numeric_only=True)
        # print(corr_day_sec)
        
        result[f'DHLD_{session}_{period}_{threshold}'] = corr_matrix.iloc[1:, 0]
        # print(threshold)
        # print(result)
        result.round(5).to_csv(f'corr_{session}_{period}.csv')

In [ ]:
sessions = ['day', 'night']
periods = ['min', 'sec']
delay = (1, 60)

# Read tick data
ticks_filename = 'TXF_tick_startFrom20240801.csv'
ticks = pd.read_csv(ticks_filename, index_col=0)
ticks = remove_duplicates(ticks)
ticks = ticks['close']
ticks.index = pd.to_datetime(ticks.index)

for session in sessions:
    for period in periods:
        correlation(ticks, session, period, 20, delay)